## Required Libraries

In [ ]:
!pip install transformers
!pip install torch
!pip install transformers[torch] accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.5.40-py3-none

## Import Dataset

In [ ]:
import pandas as pd

df = pd.read_csv("00-binary_binned_pipeline.csv")
print(df.head())  
print(df.columns)  

   Unnamed: 0  group          time    s_MP    change       type    p_MP  \
0           0      2  1.711601e+12  0.1162  0.000000  precursor  0.1162   
1           1      4  1.711602e+12  0.1155 -0.002509  precursor  0.1156   
2           2      6  1.711602e+12  0.1152 -0.001823  precursor  0.1150   
3           3      8  1.711602e+12  0.1149  0.000000  precursor  0.1149   
4           4     10  1.711603e+12  0.1150  0.000000  precursor  0.1150   

   precursor_buy_cap_pct_change  precursor_ask_cap_pct_change  \
0                     -0.009930                      0.000497   
1                      0.005317                     -0.000513   
2                     -0.006569                      0.000585   
3                      0.003920                     -0.000026   
4                     -0.005666                     -0.000039   

   precursor_bid_vol_pct_change  ...  length.1  sum_change.1  max_surge_mp.1  \
0                     -0.003699  ...       1.0      0.000606          0.1155  

## Important Columns to keep

In [ ]:
keepable = ['precursor_buy_cap_pct_change',
            'precursor_ask_cap_pct_change',
            'precursor_bid_vol_pct_change',
            'precursor_ask_vol_pct_change',
            'sum_change','length','label']
df = df[keepable]

## Splitting Data set for model Implementation

In [ ]:
# Split the data into features and labels
X = df.drop(columns=['label'])
y = df['label']

# Convert to PyTorch tensors
import torch
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Convert each row into a string format suitable for BERT
def row_to_sentence(row):
    return ' '.join([f"{col}:{val}" for col, val in row.items()])

X_train_sentences = X_train.apply(row_to_sentence, axis=1).tolist()
X_val_sentences = X_val.apply(row_to_sentence, axis=1).tolist()


## Pretrained DistilBERT Model Tokenization

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(X_train_sentences, truncation=True, padding=True)
val_encodings = tokenizer(X_val_sentences, truncation=True, padding=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

class TabularDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TabularDataset(train_encodings, y_train.tolist())
val_dataset = TabularDataset(val_encodings, y_val.tolist())


## Auto Model For Sequence Classification Using DistilBert


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Load a pretrained model and tokenizer
model_name = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=6,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_strategy='epoch',
    weight_decay=0.01,
    disable_tqdm=False
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Performance Evaluation Metrics

In [ ]:
# Import metrics for evaluation
from sklearn.metrics import accuracy_score, f1_score

# Define a function to compute evaluation metrics
def compute_metrics(pred):
    # Extract true labels from the predictions
    labels = pred.label_ids

    # Extract predicted labels by finding the index of the maximum logit
    preds = pred.predictions.argmax(-1)

    # Compute the F1 score (weighted average) between true and predicted labels
    f1 = f1_score(labels, preds, average="weighted")

    # Compute the accuracy score between true and predicted labels
    acc = accuracy_score(labels, preds)

    # Return a dictionary containing the computed accuracy and F1 score
    return {"accuracy": acc, "f1": f1}


## Training

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train();



Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.059161,0.989297,0.983974
2,No log,0.062962,0.989297,0.983974
3,No log,0.058926,0.989297,0.983974
4,No log,0.058924,0.989297,0.983974
5,No log,0.057766,0.989297,0.983974
6,No log,0.057489,0.989297,0.983974


## Evaluating The results from Training

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


Evaluation results: {'eval_loss': 0.05916079878807068, 'eval_accuracy': 0.9892966360856269, 'eval_f1': 0.9839737487277489, 'eval_runtime': 2.1903, 'eval_samples_per_second': 298.594, 'eval_steps_per_second': 5.022, 'epoch': 6.0}


## Prediction on Valuation Dataset

In [ ]:
output_preds=trainer.predict(val_dataset)
print(output_preds.metrics)

{'test_loss': 0.05916079878807068, 'test_accuracy': 0.9892966360856269, 'test_f1': 0.9839737487277489, 'test_runtime': 2.1369, 'test_samples_per_second': 306.048, 'test_steps_per_second': 5.148}
